In [1]:
import os
import yaml
import logging
import torch
import random
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import albox as alb
from tqdm import tqdm
from albox.core import Experiment
from albox.core import ExperimentSaver
from albox.models import TorchModel
device = "cuda" if torch.cuda.is_available() else "cpu"
import torchvision.models as models

/home/arun/anaconda3/envs/arun/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/arun/anaconda3/envs/arun/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/arun/anaconda3/envs/arun/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/arun/anaconda3/envs/arun/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is dep

In [2]:
from platform import python_version

print(python_version())

3.6.10


In [4]:
torch.__version__

'1.4.0'

In [3]:
import torch.backends.cudnn as cudnn
import torch.optim
from tensorboardX import SummaryWriter
import _init_paths
import models
import datasets
from config import config
from config import update_config
from core.criterion import CrossEntropy, OhemCrossEntropy
from core.function import train, validate
from utils.modelsummary import get_model_summary
from utils.utils import create_logger, FullModel


RuntimeError: Error building extension 'inplace_abn': [1/4] c++ -MMD -MF inplace_abn_cpu.o.d -DTORCH_EXTENSION_NAME=inplace_abn -DTORCH_API_INCLUDE_EXTENSION_H -isystem /home/arun/anaconda3/envs/arun/lib/python3.6/site-packages/torch/include -isystem /home/arun/anaconda3/envs/arun/lib/python3.6/site-packages/torch/include/torch/csrc/api/include -isystem /home/arun/anaconda3/envs/arun/lib/python3.6/site-packages/torch/include/TH -isystem /home/arun/anaconda3/envs/arun/lib/python3.6/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /home/arun/anaconda3/envs/arun/include/python3.6m -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++11 -O3 -c /home/arun/ALBox/examples/pytorch_segmentation/cityscapes_HRnet/models/sync_bn/inplace_abn/src/inplace_abn_cpu.cpp -o inplace_abn_cpu.o
FAILED: inplace_abn_cpu.o 
c++ -MMD -MF inplace_abn_cpu.o.d -DTORCH_EXTENSION_NAME=inplace_abn -DTORCH_API_INCLUDE_EXTENSION_H -isystem /home/arun/anaconda3/envs/arun/lib/python3.6/site-packages/torch/in